# **Welcome to Bittensor!**

(This notebook is supposed to run with pip-installed version of bittensor)

Bittensor is a decentralized market that enables individuals to monetize intelligence production from any computer anywhere in the world. Intelligence production is validated by the other peers in the network and rewarded through token inflation. Consumers stake this currency to gain access to the produced knowledge. Bittensor is collectively-run, open-source, and open-access.

For more info...

* [Visit our website](https://www.bittensor.com/)
* [Read our paper](https://arxiv.org/abs/2003.03917)
* [Check out the code](https://github.com/opentensor/BitTensor)
* [Talk to us!](https://discord.gg/3rUr6EcvbB)

### **Some basic terminologies**
Before moving forward, know these terms!

1. Tau: The currency name.
2. Neuron/node/peers/endpoint: Server that mines Tau, each server owns a machine learning model, which we call nucleus.
3. Network/chain: The whole group of people that miners Tau.
4. Weight: The scoring that other neurons in the network gives you. 
5. Stake: The higher the stake, the more Tau you own. It is also determined by your weight and the time you stay in the network.

### **Structure of this tutorial**
1. [Quick start!](#quickStart) 
3. [Wallet](#wallet)
2. [Metagraph](#metagraph)
6. [Subtensor](#subtensor)
4. [Dendrite](#dendrite)
5. [Axon](#axon)



### **Let's get the bittensor package ready!**

In [ ]:
# ---- install the package ----
! pip install bittensor

In [1]:
# ---- import ----
import bittensor
import torch

# ---- Enable logging from bittensor ----
bittensor.logging(debug = False)

2021-08-19 09:51:10.697 |     SUCCESS      | Set debug:          OFF
2021-08-19 09:51:10.697 |     SUCCESS      | Set trace:          OFF
2021-08-19 09:51:10.697 |     SUCCESS      | Set record log:     OFF


<a id='quickStart'></a>
# **Quick start!** 

#### **Client side**

On the client side, we can send request for a representation of our data from each of the chosen node.

In [2]:
# ---- set up a wellet ----
wallet = bittensor.wallet().create()

# ---- sync to our metagraph (this will be further explained) ----
graph = bittensor.metagraph().sync()

# ---- querying for representation by calling dendrite ----
representations, _ = bittensor.dendrite( wallet = wallet ).forward_text (
    endpoints = graph.endpoints,
    inputs = "The quick brown fox jumped over the lazy dog"
)
# ---- checking out the representations ----
# representations = Tensor with shape (number of nodes online, 9, 512)

2021-08-18 16:49:10.882 |     SUCCESS      | Loaded coldkey.pub: 0x72295375bdd9a3c85a2df216644d31ef08ec3988c98022c09fcffd9ac01aaf12
2021-08-18 16:49:10.885 |     SUCCESS      | Loaded hotkey:      0xfabd758dc0e148bfb86813cb451acbec27d96e19690cb1e1072036ac5510746c


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  2.54it/s]


### Server side

On the server side, we get to mine Tau using our own model.

In [2]:
from transformers import BertModel, BertConfig

# ---- set up a wallet ----
wallet = bittensor.wallet().create()

# ---- set up the forward function ----
model = BertModel(BertConfig())

def forward ( pubkey, inputs_x, modality ):
    return model( inputs_x ).narrow(2, 0, bittensor.__network_dim__)

# ---- subscribe to axon, that handles forward and backward request from other neurons ----
axon = bittensor.axon (
    wallet = wallet,
    forward_text = forward,
).start().subscribe()

2021-08-19 09:52:27.806 |     SUCCESS      | Loaded coldkey.pub: 0x72295375bdd9a3c85a2df216644d31ef08ec3988c98022c09fcffd9ac01aaf12
2021-08-19 09:52:27.807 |     SUCCESS      | Loaded hotkey:      0xfabd758dc0e148bfb86813cb451acbec27d96e19690cb1e1072036ac5510746c
2021-08-19 09:52:34.224 |     SUCCESS      | Axon Stopped:       [::]:8091
2021-08-19 09:52:34.227 |     SUCCESS      | Axon Started:       [::]:8091
2021-08-19 09:52:34.770 |     SUCCESS      | External IP:        74.14.25.112
2021-08-19 09:52:35.459 |     SUCCESS      | Subscribed with     ip: 74.14.25.112, port: 8091, modality: 0, hotkey: 0xfabd758dc0e148bfb86813cb451acbec27d96e19690cb1e1072036ac5510746c, coldkey: 0x72295375bdd9a3c85a2df216644d31ef08ec3988c98022c09fcffd9ac01aaf12


<a id='wallet'></a>
# **Wallet**

A wallet has the following properties.
* name
* coldkey: Used to store, transfer, and stake tokens. It is "cold" because it is not loaded into the miner and remains encrypted on the device. 
* hotkey: Used by the miner to subscribe and set weights. It is "hot" because it is loaded into the running software (which can be insecure). It does not have permission to move funds.

NOTE: Remember to save the mnemonic of the cold key and hot key for regenerating your password or hot key.

In [33]:
# ---- initialize a wallet object ----
# if you want to creat a new wallet, choose a new wallet name, else just choose an existing wallet name 

wallet = bittensor.wallet(name = 'wallet-name')
wallet.create()

2021-08-18 16:54:58.596 |     SUCCESS      | Loaded coldkey.pub: 0x4e8c74eeba33a71f738f5eb7fb94bd768bbbeca01c1857fd79b4e47277bb646d
2021-08-18 16:54:58.597 |     SUCCESS      | Loaded hotkey:      0x4a99ff91d2813eb74f9f5e458e66c4d55fcb890bb23f2e87d37f24798b1f3501


<a id='metagraph'></a>
# **Metagraph** 
Metagraph is an object that maintains the chain state as a torch.nn.Module. First, setp up the metagraph object.

In [ ]:
# ---- Initializing the metagraph object ----
meta = bittensor.metagraph()

# ---- Synchronise states in meta graph with the akatsuki network, aka the chain----
meta = meta.sync()

print("---- Metagraph keys ----\n\n", meta.state_dict().keys())

### **Block info**
* n: the number of neurons in the network
* block: the block id 
* tau (T): the token inflation rate per block

In [11]:
print("---- Block info ----\n")
print("n:\t\t", meta.n.item())
print("block ID:\t", meta.block.item())
print("tau:\t\t", meta.tau[0])

---- Block info ----

n:		 tensor(37)
block ID:	 tensor(416893)
tau:		 tensor(0.5000)


### **Neurons' info**
* uids: the user id of each neurons
* last_update: the last time the the users emit to the chain
* endpoints: the encoded ip addresses, hotkey, coldkey of other neurons. Can be retrieved by bittensor.endpoint.from_tensor().

In [21]:
print("---- Neurons' info (of the first 5 neurons) ----\n")
print("uids :\t", meta.uids[0:5])
print("last_update :\t", meta.last_update[0:5])
print("\nendpoint (of the first neuron):\n\n", meta.endpoints[0])
print("\nip address/hotkey/coldkey (of the first neuron): \n\n", bittensor.endpoint.from_tensor(meta.endpoints[0]))

print("\nhotkeys: \n\n", meta.hotkeys[0:5])
print("\ncoldkeys: \n\n", meta.coldkeys[0:5])
print("\naddresses: \n\n", meta.addresses[0:5])

---- Neurons' info (of the first 5 neurons) ----

uids :	 tensor([0, 1, 2, 3, 4])
lastemit :	 tensor([ 16106,  16171, 378258,  31431,  31790])

endpoint (of the first neuron):

 tensor([123,  34, 117, 105, 100,  34,  58,  32,  48,  44,  32,  34, 104, 111,
        116, 107, 101, 121,  34,  58,  32,  34,  53,  67, 118, 102,  69, 102,
         86,  89,  86,  81, 120,  66, 122, 112,  68,  87,  98,  98,  55,  72,
         84,  53, 118, 116,  70,  87, 107,  65, 102, 107,  54, 120,  53, 104,
         57,  86,  55, 100, 111,  66,  71, 115, 116, 104, 104, 114, 116, 111,
         34,  44,  32,  34, 105, 112,  34,  58,  32,  34,  57,  57,  46,  50,
         51,  56,  46,  49,  51,  54,  46,  53,  54,  34,  44,  32,  34, 105,
        112,  95, 116, 121, 112, 101,  34,  58,  32,  52,  44,  32,  34, 112,
        111, 114, 116,  34,  58,  32,  56,  48,  57,  49,  44,  32,  34,  99,
        111, 108, 100, 107, 101, 121,  34,  58,  32,  34,  53,  69, 118, 112,
         52,  78, 102, 116,  87,  70,  52,

### **Neuron's scorings**
* weight (W): the weight of of the network, which is an n*n matrix. W_{i,j} indicates the locally calculated score from uid[i] to uid[j]. 
* rank (R): R = W<sup>T</sup> &middot; S. It also controls the priority of message to be read by peers.
* trust (T): T = $\sigma$ (C <sup>T</sup> S), where C is the connectivity matrix of size n*n. C<sub>i,j</sub> indicates the number of expectation in time that peer i reach peer j.
* incentive (I): I = W<sup>T</sup> S &middot; $\sigma$ (C <sup>T</sup> S)
* stake (S): the stake of each neuron, updated with function S<sub>t+1</sub> = S<sub>t</sub> + (&tau; R ||S||) / ||R|| 

In [10]:
print("---- Neurons' scoring (of and within the first 5 neurons) ----\n")
print("Weight:\n", meta.W[0:5, 0:5])
print("Stake:\t", meta.S[0:5])
print("Rank:\t", meta.R[0:5])
print("Incentive:\t", meta.I[0:5])

---- Neurons' scoring (of and within the first 5 neurons) ----

Weight:
 tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])
Stake:	 tensor([0., 0., 0., 0., 0.])
Rank:	 tensor([0., 0., 0., 0., 0.])
Incentive:	 tensor([0., 0., 0., 0., 0.])


### **Saving and loading the metagraph** 

In [9]:
meta.save()
meta.load()

meta.save_to_path('~/.bittensor/', 'network-name.pt')
meta.load_from_path('~/.bittensor/network-name.pt')

Metagraph (37,406630,akatsuki)

### **Use Cases**

In [9]:
# TODO: maybe list more use cases

# ---- getting the endpoints with stake at the top 90% ----
meta.endpoints[meta.S > torch.quantile(meta.S, 0.9)]

# ---- querying the stake of a neuron from its pubkey(hotkey) ----
# stake_of_caller = meta.S[meta.hotkeys.index(pubkey)]

[(tensor([123,  34, 117, 105, 100,  34,  58,  32,  51,  53,  44,  32,  34, 104,
          111, 116, 107, 101, 121,  34,  58,  32,  34,  53,  70,  52, 101, 119,
           87,  51,  51,  54, 121,  71,  71, 121, 103, 105,  75, 105,  99,  68,
          111,  99,  71,  50,  56,  88, 107, 111,  83, 111, 116,  67, 120,  70,
          109, 101,  99,  81, 109,  85,  53,  68,  83,  56, 110,  85,  50,  80,
           76,  34,  44,  32,  34, 105, 112,  34,  58,  32,  34,  49,  52,  56,
           46,  49,  48,  50,  46,  49,  49,  51,  46,  49,  51,  50,  34,  44,
           32,  34, 105, 112,  95, 116, 121, 112, 101,  34,  58,  32,  52,  44,
           32,  34, 112, 111, 114, 116,  34,  58,  32,  56,  48,  57,  49,  44,
           32,  34,  99, 111, 108, 100, 107, 101, 121,  34,  58,  32,  34,  53,
           70, 115,  86, 109,  67,  75,  86,  68, 118, 107,  85, 118,  88,  82,
           52,  50,  99, 107, 100, 105,  57,  71, 120, 109, 103,  85,  56,  67,
           50, 122,  81, 118,  75, 109, 

<a id='subtensor'></a>
# **Subtensor**

It handles the interactions with the subtensor chain.

In [72]:
# ---- initialize ----
subtensor = bittensor.subtensor()

# ---- Subscribes an bittensor endpoint to the substensor chain. ----
subtensor.subscribe(wallet, axon.external_ip, axon.external_port, 0)

# ---- shall we include these functions as well? ----

# subtensor.connect()
# subtensor.set_weights()
# subtensor.get_balances()

2021-08-19 10:56:31.569 |     SUCCESS      | Subscribed with     ip: 74.14.25.112, port: 8091, modality: 0, hotkey: 0xfabd758dc0e148bfb86813cb451acbec27d96e19690cb1e1072036ac5510746c, coldkey: 0x72295375bdd9a3c85a2df216644d31ef08ec3988c98022c09fcffd9ac01aaf12


True

<a id='dendrite'></a>
# **Dendrite**

The dendrite object is for questing representation of your data from other neurons.

In [53]:
# the projected size of representation
bittensor.__network_dim__

512

In [54]:
# ---- initializing a dendrite object ----
# note: it is not required to attach a wallet to the dendrite object 
dend = bittensor.dendrite(wallet = wallet)

In [57]:
# ---- specify the endpoints that you want to send request to ----
# here we select the top 10% endpoints that has the most stake 
endpoints = meta.endpoints[meta.S > torch.quantile(meta.S, 0.9)]

# ---- retrieving the represenataions from other 
representation = dend.forward_text(endpoints, "The quick brown fox jumped over the lazy dog" )

In [14]:
# ---- checking on the representation ----
representation[0]

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]], grad_fn=<DendriteBackward>),)

<a id='axon'></a>
# **Axon**

An axon object that handles forward and backward request from other neurons.

In [15]:
# ---- first, define a model and a forward function to be passed to the Axon object ----
# the resulted model representation has to be narrowed to our network_dim 

from transformers import BertModel, BertConfig
model = BertModel(BertConfig())

def forward ( pubkey, inputs_x, modality ):
    return model( inputs ).narrow(2, 0, bittensor.__network_dim__)

In [63]:
# ---- initialize the axon object ----
# ---- start the axon object, but other neurons still can't find you!!! ----
# ---- subscribe the axon object to the network. Okay! other neurons can see you now! ---- 
axon_ = bittensor.axon (
    wallet = wallet,
    forward_text = forward,
).start().subscribe()

2021-08-19 10:36:26.515 |     SUCCESS      | Axon Stopped:       [::]:8091
2021-08-19 10:36:26.518 |     SUCCESS      | Axon Started:       [::]:8091
2021-08-19 10:36:26.930 |     SUCCESS      | External IP:        74.14.25.112
2021-08-19 10:36:27.597 |     SUCCESS      | Subscribed with     ip: 74.14.25.112, port: 8091, modality: 0, hotkey: 0xfabd758dc0e148bfb86813cb451acbec27d96e19690cb1e1072036ac5510746c, coldkey: 0x72295375bdd9a3c85a2df216644d31ef08ec3988c98022c09fcffd9ac01aaf12


In [64]:
# Bug?
axon_ = bittensor.axon (
    wallet = wallet,
    forward_text = forward,
)
axon.start()
axon.subscribe()

2021-08-19 10:36:29.630 |     SUCCESS      | Axon Stopped:       [::]:8091


ValueError: Cannot start already-started server!

In [58]:
# ---- check your subscribed ip and port ----
print(f"{axon.external_ip}:{axon.external_port}" )

74.14.25.112:8091
